In [ ]:

!pip install transformers
!pip install torch
!pip install sentencepiece


In [ ]:
!unzip /content/fine_tuned_t5_fire_and_blood.zip


In [3]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer


In [ ]:
model_path = 'fine_tuned_t5_fire_and_blood'  # Change this path if necessary
tokenizer = T5Tokenizer.from_pretrained('t5-base')

model = T5ForConditionalGeneration.from_pretrained(model_path)


In [11]:
def generate_question(text):
    input_text = "generate_question: " + text
    input_tensor = tokenizer.encode(input_text, return_tensors='pt')
    output_tensor = model.generate(input_tensor, max_length=100, num_return_sequences=1)
    output_text = tokenizer.decode(output_tensor[0], skip_special_tokens=True)
    return output_text

def generate_answer(text, question):
    input_text = f"generate_answer: context: {text} question: {question}"
    input_tensor = tokenizer.encode(input_text, return_tensors='pt')
    output_tensor = model.generate(input_tensor, max_length=100, num_return_sequences=1)
    output_text = tokenizer.decode(output_tensor[0], skip_special_tokens=True)
    return output_text





In [12]:
text = "House Targaryen was a noble family of Valyrian descent who escaped the Doom. They lived for centuries on the island of Dragonstone until Aegon the Conqueror and his sisters rode their dragons to conquer the Seven Kingdoms."

question = generate_question(text)
print("Question:", question)

answer = generate_answer(text, question)
print("Answer:", answer)


Question: generate_question: House Targaryen noble family Valyrian descent escaped Doom. lived centuries island Dragonstone Aegon Conqueror sisters rode dragon conquer Seven Kingdoms.
Answer: House Targaryen noble family Valyrian descent escaped Doom


In [10]:
text = "House Targaryen was a noble family of Valyrian descent who escaped the Doom. They lived for centuries on the island of Dragonstone until Aegon the Conqueror and his sisters rode their dragons to conquer the Seven Kingdoms."

question, answer = generate_qa(text)
print("Question:", question)
print("Answer:", answer)


Question: Failed to generate a question and answer.
Answer: 


In [13]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [14]:
def generate_question(text):
    input_text = "generate_question: " + text
    input_tensor = tokenizer.encode(input_text, return_tensors='pt')
    output_tensor = model.generate(input_tensor, max_length=100, num_return_sequences=1)
    output_text = tokenizer.decode(output_tensor[0], skip_special_tokens=True)
    return output_text

def generate_answer(text, question):
    input_text = f"generate_answer: context: {text} question: {question}"
    input_tensor = tokenizer.encode(input_text, return_tensors='pt')
    output_tensor = model.generate(input_tensor, max_length=100, num_return_sequences=1)
    output_text = tokenizer.decode(output_tensor[0], skip_special_tokens=True)
    return output_text


In [15]:
text = "House Targaryen was a noble family of Valyrian descent who escaped the Doom. They lived for centuries on the island of Dragonstone until Aegon the Conqueror and his sisters rode their dragons to conquer the Seven Kingdoms."

question = generate_question(text)
print("Question:", question)

answer = generate_answer(text, question)
print("Answer:", answer)


Question: True
Answer: True


In [ ]:
!pip install openai


In [45]:
import openai

api_key = "api key"
openai.api_key = api_key


In [46]:
def generate_multiple_qa_gpt3(text, num_qa_pairs=3, model="text-davinci-002"):
    prompt = f"{text}\n\nGenerate {num_qa_pairs} question-answer pairs:\n"
    for i in range(1, num_qa_pairs + 1):
        prompt += f"Q{i}: {{Question {i}}}\nA{i}: {{Answer {i}}}\n"
    
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        max_tokens=200,
        n=1,
        stop=None,
        temperature=0.5,
    )

    generated_text = response.choices[0].text.strip()
    return generated_text



In [47]:
!pip install pandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
import pandas as pd


In [49]:
# Create an empty DataFrame to store the results
result_columns = ["text", "generated_qa_pairs"]
result_data = pd.DataFrame(columns=result_columns)


In [ ]:
# Read the CSV file
csv_file_path = "paragraphs (1).csv"
data = pd.read_csv(csv_file_path)

# Iterate through the rows and process each text
for index, row in data.iterrows():
    text = row['Paragraph']
    
    # Call the generate_multiple_qa_gpt3 function or any other function you want to use with the text
    num_qa_pairs = 3
    generated_qa_pairs = generate_multiple_qa_gpt3(text, num_qa_pairs)
    #print(f"Text {index + 1}:")
    #print(text)
    #print("Generated QA pairs:")
    #print(generated_qa_pairs)
    #print("\n")
    
    # Append the result to the result_data DataFrame
    result_data = result_data.append({"text": text, "generated_qa_pairs": generated_qa_pairs}, ignore_index=True)


In [51]:
# Save the result_data DataFrame to a new CSV file
output_csv_file_path = "output_csv_file_path_here.csv"
result_data.to_csv(output_csv_file_path, index=False)


In [59]:
# Print the first few generated QA pairs from the result_data DataFrame
for index, row in result_data.head().iterrows():
    print(f"Generated QA pairs for text {index + 1}:")
    print(row['generated_qa_pairs'])
    print("\n")


In [ ]:
# Read the CSV file containing the generated question-answer pairs
generated_qa_pairs_file_path = "OpenAI_Generated_Q&A's.csv"
result_data = pd.read_csv(generated_qa_pairs_file_path)

# Create an empty DataFrame to store the results
qa_data_columns = ["text", "question", "answer"]
qa_data = pd.DataFrame(columns=qa_data_columns)

# Iterate through the rows and process each text
for index, row in result_data.iterrows():
    text = row['text']
    generated_qa_pairs = row['generated_qa_pairs']

    # Extract individual questions and answers
    qa_pairs = re.findall(r'Q\d+: (.*?)\nA\d+: (.*?)(?=$|\nQ)', generated_qa_pairs, re.MULTILINE | re.DOTALL)

    # Append each question-answer pair to the qa_data DataFrame
    for question, answer in qa_pairs:
        qa_data = qa_data.append({"text": text, "question": question, "answer": answer}, ignore_index=True)

# Save the qa_data DataFrame to a new CSV file
output_csv_file_path = "qa_output.csv"
qa_data.to_csv(output_csv_file_path, index=False)
